## 1. 데이터 전처리

In [2]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

빠른 학습을 위해 학습 데이터와 테스트 데이터를 2:8 비율로 분할합니다. 이 값을 변경하고자 하는 분은 test_ratio의 값을 변경하세요.

In [4]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

# 2. system_message 정의
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""

# 3. 원본 데이터의 type별 분포 출력 
print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

# 4. train/test 분할 비율 설정 (0.5면 5:5로 분할)
test_ratio = 0.8

train_data = []
test_data = []

# 5. type별로 순회하면서 train/test 데이터 분할
for type_name in set(dataset['type']):
    # 현재 type에 해당하는 데이터의 인덱스만 추출
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]
    
    # test_ratio에 따라 test 데이터 개수 계산 
    test_size = int(len(curr_type_data) * test_ratio)
    
    # 현재 type의 데이터를 test_ratio 비율로 분할하여 추가
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

# 6. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # 검색 결과를 문서1, 문서2... 형태로 포매팅
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])
    
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

# 7. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 8. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

# 9. 분할된 데이터의 type별 분포 출력
print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

README.md:   0%|          | 0.00/909 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

원본 데이터의 type 분포:
synthetic_question: 497
mrc_question: 491
mrc_question_with_1_to_4_negative: 296
paraphrased_question: 196
no_answer: 404

전체 데이터 분할 결과: Train 380개, Test 1504개

학습 데이터의 type 분포:
synthetic_question: 100
mrc_question: 99
mrc_question_with_1_to_4_negative: 60
paraphrased_question: 40
no_answer: 81

테스트 데이터의 type 분포:
synthetic_question: 397
mrc_question: 392
mrc_question_with_1_to_4_negative: 236
paraphrased_question: 156
no_answer: 323


In [5]:
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: “우버(UBER)는 사람들이 이동하는 수단을 발전시켰습니다. 승객과 기사를 실시간으로 연결해 승객에게는 편리함을, 기사에게는 더 많은 효율성과 수익을 가져다 주었습니다. 2010년 6월 미국 샌프란시스코에서 처음 서비스를 시작한 뒤 현재 세계 140여개 도시에 진출했습니다. 우버는 앞으로도 계속 사람과 도시를 가깝게 이어줄 것입니다.”샌프란시스코 우버 본사에서 최근 만난 나이리 후다지안 우버 글로벌커뮤니케이션 부문장은 이렇게 강조했다. 우버는 일종의 ‘차량 예약 서비스’를 제공하는 글로벌 정보기술(IT) 기업이다. 스마트폰 애플리케이션(앱·응용프로그램·사진)을 통해 승객과 차량을 연결해 주는 사업을 하고 있다.승객을 일반택시와 연결해 주는 ‘우버택시’, 일반인이 자신의 차량으로 운송 서비스를 할 수 있도록 도와주는 ‘우버엑스’, 일종의 고급 콜택시인 ‘우버블랙’ 등의 서비스를 갖췄다. 한국에서는 지난해 8월 우버코리아가 설립돼 우버블랙 사업을 펼치고 있다.후다지안 부문장은 “우리는 더 나은 교통 서비스 위해 플랫폼을 제공하는 회사”라

In [6]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [7]:
train_dataset[0]

{'messages': [{'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 도시 농업의 경우, 세계 각지에서 지구 온난화 문제, 대기의 건조 및 미중 무역 분쟁, 한일 경제 전쟁 등의 여파에 따라 계속 발생되는 아마존 우림 산불 등과 같은 각종 자연재해 등이 지속적으로 일으키게 되면서, 기후 변화로 인한 생태계 파괴 등을 막을 수 있는 조건도 있고 자발적인 생활 패턴이 생기게 되는 대안을 제시할 수 있게 되는 이점이 있다. 그러나 도시 농업을 활성화하게 되면 숲과 같은 삼림 보호 정책은 물론 바다나 개펄 등 해양 생태계 보호 정책까지 대폭적으로 강화할 수 있는 등 다양한 니즈를 충족시킬 수 있게 된다. 기존에는 삼림 벌채 후 농경지 조성을 하였지만, 건물 최상층, 중간 피난층, 주상복합아파트 같은 주거 및 상업시설 복합 건축물의 상업 지역 부분 옥상 등지에 농지를 가꾸어져서 건물에 상주하고 있는 분들과 주거 시설 거주민들이 1년 동안 안정적으로 식량 자원을 확보할 수 있게 되는 좋은 점이 있다. 이미 EBS TV의 하나뿐인 지구라는 교양 프로그램에도 이와 같은 배경을 소재로 둔 이야기를 토대로 보면, 기후 변화 대응 체

## 2. 모델 로드 및 템플릿 적용

In [8]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-7B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [9]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 도시 농업의 경우, 세계 각지에서 지구 온난화 문제, 대기의 건조 및 미중 무역 분쟁, 한일 경제 전쟁 등의 여파에 따라 계속 발생되는 아마존 우림 산불 등과 같은 각종 자연재해 등이 지속적으로 일으키게 되면서, 기후 변화로 인한 생태계 파괴 등을 막을 수 있는 조건도 있고 자발적인 생활 패턴이 생기게 되는 대안을 제시할 수 있게 되는 이점이 있다. 그러나 도시 농업을 활성화하게 되면 숲과 같은 삼림 보호 정책은 물론 바다나 개펄 등 해양 생태계 보호 정책까지 대폭적으로 강화할 수 있는 등 다양한 니즈를 충족시킬 수 있게 된다. 기존에는 삼림 벌채 후 농경지 조성을 하였지만, 건물 최상층, 중간 피난층, 주상복합아파트 같은 주거 및 상업시설 복합 건축물의 상업 지역 부분 옥상 등지에 농지를 가꾸어져서 건물에 상주하고 있는 분들과 주거 시설 거주민들이 1년 동안 안정적으로 식량 자원을 확보할 수 있게 되는 좋은 점이 있다. 이미 EBS TV의 하나뿐인 지구라는 교양 프로그램에도 이와 같은 배경을 소재로 둔 이야기를 토대로 보면, 기후 변화 대응 체계 해소 취지를 앞장서는 여건이 마련

## 3. LoRA와 SFTConfig 설정

In [10]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. `0.1`로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. `"none"`으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. `"all"` 또는 `"lora_only"`와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 `"q_proj"`와 `"v_proj"`를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type`: LoRA가 적용되는 작업 유형을 지정합니다. `"CAUSAL_LM"`은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 `"SEQ2SEQ_LM"`(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [11]:
args = SFTConfig(
    output_dir="qwen2-7b-rag-ko",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

`output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜹니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [12]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

`collate_fn(batch)` 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 레이블 데이터를 초기화한 다음, 특정 토큰 패턴(<|im_start|>assistant 이후부터 <|im_end|>까지)을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [13]:
# 최대 길이
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 4328])
어텐션 마스크 형태: torch.Size([1, 4328])
레이블 형태: torch.Size([1, 4328])


In [14]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 138520, 19391, 143604, 129264, 130650, 382, 13146, 48431, 20401, 66790, 29326, 131193, 17877, 125686, 125548, 139713, 624, 16, 13, 138520, 53680, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 16186, 139713, 624, 17, 13, 85322, 77226, 98801, 19391, 130768, 130213, 17877, 143604, 16186, 125476, 34395, 53900, 21329, 95577, 139713, 624, 18, 13, 138520, 19391, 128605, 143603, 12802, 85322, 77226, 98801, 19391, 130671, 32290, 85322, 77226, 98801, 126377, 330, 33883, 64795, 138520, 93, 19391, 128605, 130213, 12802, 136673, 1189, 5140, 45881, 34395, 143604, 16186, 139713, 624, 19, 13, 143604, 47836, 53618, 142976, 139236, 18411, 142616, 82190, 53435, 40853, 129549, 53435, 125068, 17877, 140174, 128836, 32290, 5140, 240, 97, 19391, 36330, 250, 125746, 16560, 23084, 126402, 83634, 17380, 94613, 139236, 84621, 47324, 18411, 129624, 20487, 139713, 13, 95617, 18411, 129901, 26698

In [15]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.

---

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.  
사용자가 **"안녕하세요, 오늘 날씨는 어떤가요?"**라고 물었고,  
모델은 **"안녕하세요! 오늘 날씨는 맑고 화창합니다."**라고 응답해야 한다고 합시다.

이 데이터를 학습하기 위해 먼저 전체 대화 데이터를 정수로 인코딩합니다.  
토크나이저는 대화의 구조를 구분하기 위해 `<|im_start|>`와 `<|im_end|>`을 사용하여 정수 시퀀스를 생성한다고 가정해봅시다.  
(실제로는 LLM 템플릿이 이보다는 복잡함을 기억하고 혼동하지 맙시다.)
정수 시퀀스는 다음과 같이 구성될 수 있습니다.  

---
<|im_start|>user 안녕하세요, 오늘 날씨는 어떤가요?<|im_end|>  
<|im_start|>assistant 안녕하세요! 오늘 날씨는 맑고 화창합니다.<|im_end|>
---

이를 정수로 변환하면 다음과 같습니다.  
`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`  
모델이 예측해야 하는 부분은 `assistant`의 응답인 "안녕하세요! 오늘 날씨는 맑고 화창합니다."입니다. 따라서, `labels`는 다음과 같이 설정됩니다.

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`  

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져 모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다. 이를 통해 모델은 불필요한 영역을 학습하지 않고, 필요한 응답 영역에만 집중할 수 있습니다.


## 5. 학습

In [16]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [17]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.586400
20,0.556000
30,0.503000
40,0.388600
50,0.436800
60,0.375900
70,0.385800
80,0.448000
90,0.420500
100,0.368800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 6. 테스트 데이터 준비

실제 모델에 입력을 넣을 때에는 입력의 뒤에 '<|im_start|>assistant'가 부착되어서 넣는 것이 좋습니다. 그래야만 모델이 바로 답변을 생성합니다.

In [18]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [22]:
print(prompt_lst[42])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: ‘관피아 논란’에도 불구하고 일찌감치 민간으로 옮겨 전문성과 능력을 인정받는 관료 출신들도 적지 않다. 대표적인 인물이 권용원 키움증권 사장이다. 권 사장은 1999년까지 산업자원부 산업기술개발과장(기술고시 21회)을 지내다가 이듬해 다우그룹 전략경영실장으로 이직했다. 다우그룹에서 신사업을 전담한 그는 신생 온라인 증권사였던 키움증권을 대형 증권사로 키워냈다.금융투자업계에선 전병조 KB투자증권 부사장이 눈에 띈다. 6년 전 기획재정부 관료(행정고시 29회)에서 기업금융(IB) 부문 전문가로 변신했다. 재정경제부 출신 이형승 케이스템셀 회장(행시 29회)은 삼성증권(이사) CJ그룹(소장) IBK투자증권(사장)을 거쳐 바이오회사로 갔다. 재정경제부 출신 이현승 전 SK증권 사장(행시 32회)도 AT커니(이사) 메릴린치(이사) GE에너지(사장) 등을 거쳤다.대기업 쪽에도 많다. 박영춘 SK 전무(행시 31회)는 금융위원회 금융정책과장을 거쳐 청와대 비상경제상황실 금융팀장을 지낸 금융 전문가로 SK그룹의 전략업무를 맡고 있다. 박 전무와 행시 동기인 문홍성 두산 부사장은 기획재정부 출신으

In [23]:
print(label_lst[42])


정부와 기업 간 협업의 사례와 그 결과는 다음과 같습니다.

1. **(주)이주와 정부의 협업**:
   - **사례**: 대구지역 섬유업체인 (주)이주는 2006년 정부로부터 부품소재전문기업으로 인정받고, 2008년에는 기술혁신형 중소기업으로 선정되었습니다. 이 회사는 정부 과제를 수주한 뒤 대구·경북 지역의 전문 연구소와 제휴를 맺어 연구 과제를 수행했습니다.
   - **결과**: 이러한 협업을 통해 (주)이주는 좋은 조건으로 자금을 사용할 수 있게 되었고, 대구시장 표창과 교육과학기술부 장관 표창을 받는 등 여러 성과를 거두었습니다. 이는 섬유업이 지식산업으로 탈바꿈할 수 있다는 가능성을 보여주는 사례로 평가받고 있습니다[[ref2]].

2. **정주영 현대그룹 명예회장과 박정희 대통령의 협업**:
   - **사례**: 1960년대 초 현대건설이 시공하는 소양강댐 건설 방식을 두고 정주영 회장과 박정희 대통령이 협업했습니다. 정 회장은 일본의 댐 설계 전문가와의 설전에서 소양강 주변의 자원을 이용한 사력댐 건설을 주장했고, 박 대통령은 정 회장의 손을 들어주었습니다.
   - **결과**: 이 협업의 결과로 공사비를 30% 절감할 수 있었고, 더 안전한 댐을 건설할 수 있었습니다[[ref5]].

3. **코오롱 창업주 이원만 회장과 박정희 대통령의 협업**:
   - **사례**: 이원만 회장은 1963년 박정희 대통령을 설득해 서울 구로동 수출공업단지 건설을 이끌어냈습니다. 그는 한국 근로자들의 손재주를 활용해 스테인리스 제품을 수출할 수 있다는 점을 강조했습니다.
   - **결과**: 이 협업을 통해 구로동 수출공업단지가 건설되었고, 이는 한국의 수출 산업 발전에 기여했습니다[[ref5]].

이와 같은 사례들은 정부와 기업 간의 협업이 기업의 성장과 국가 경제 발전에 중요한 역할을 할 수 있음을 보여줍니다.<|im_end|>



## 7. 파인 튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"qwen2-7b-rag-ko/checkpoint-285"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.

In [24]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [25]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [28]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [29]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [30]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
스마트 팩토리 솔루션은 식품 제조업체와 철강업체에 다양한 방식으로 적용되고 있습니다.

### 식품 제조업체
#### 진성에프엠
- **사례**: 에어릭스는 진성에프엠 화성 공장에 스마트팩토리 IoT 솔루션을 구축했습니다. 이 솔루션은 급기·배기 팬의 온도 및 진동을 모니터링하는 IoT 회전체 모니터링 시스템(iCMS)과 PLC 설비 모니터링 시스템을 설치하여, 공장 내 모든 설비에 대한 이상 유무를 실시간 모니터링할 수 있도록 했습니다. 또한, IoT 종합 모니터링 시스템(iTMS)을 구축하여 관리자가 중앙에서 모니터링할 수 있도록 했습니다. 이로 인해 설비 이상을 미연에 방지하고 생산 관리를 개선할 수 있게 되었습니다 [[ref3]].

### 철강업체
#### 현대제철
- **사례**: 현대제철은 현대차그룹의 첫 냉연공장을 통합하고, 철강 사업을 확장했습니다. 이로 인해 쇳물 생산부터 자동차 외장재로 쓰이는 철강 제품 생산에 이르는 전 과정을 통합할 수 있게 되었습니다. 이 결과, 제품 포트폴리오가 다양화되었고, 냉연제품 매출이 증가했습니다. 또한, 원가절감 효과와 무형의 시너지가 발생하여 재무실적이 개선되었습니다. 예를 들어, 코일 무게(단중)가 증가하고, 작업의 안정성과 효율성이 높아졌습니다 [[ref4]].

이와 같이, 스마트 팩토리 솔루션은 식품 제조업체와 철강업체에서 생산 과정의 효율性和 안정성을 향상시키는 데 중요한 역할을 하고 있습니다.
    label:

스마트 팩토리 솔루션은 식품 제조업체와 철강업체에 각각 다른 방식으로 적용되고 있습니다.

식품 제조업체의 경우, 스마트 팩토리 솔루션은 주로 생산 관리와 설비 모니터링에 중점을 두고 있습니다. 예를 들어, 농축액 음료 제조업체 '진성에프엠'의 화성 공장에서는 에어릭스의 스마트 팩토리 IoT 솔루션이 도입되었습니다. 이 솔루션은 급기·배기 팬의 온도 및 진동을 모니터링하는 IoT 회전체 모니터링 시스템(iCMS)과 PLC 설비 모니터링 시스템을 포함하고 있습니다

## 8. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.

In [31]:
base_model_id = "Qwen/Qwen2-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [32]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
스마트 팩토리 솔루션은 식품 제조업체와 철강업체에 다양한 방식으로 적용되고 있습니다.

식품 제조업체에 있어서, 에어릭스(대표이사 김군호)가 농축액 음료 제조업체 '진성에프엠' 화성 공장에 스마트팩토리 IoT 솔루션을 구축하였습니다. 이 솔루션은 급기·배기 팬의 온도 및 진동을 모니터링하는 IoT 회전체 모니터링 시스템(iCMS)과 PLC 설비 모니터링 시스템을 설치함으로써 설비 이상을 미연에 방지하고 생산 관리를 개선하는데 도움을 주고 있습니다. 또한, IoT 종합 모니터링 시스템(iTMS)을 구축하여 관리자가 중앙에서 현장 내 모든 설비에 대한 이상 유무를 실시간 모니터링 할 수 있도록 지원하고 있습니다.

철강업체에 있어서, 현대제철은 냉연사업 인수를 통해 새로운 날개를 달았습니다. 이로 인해 제품 포트폴리오가 다양화되었고, 재무실적도 개선되었습니다. 특히, 쇳물 생산부터 자동차강판 생산에 이르는 전 과정을 통합함으로써 원가절감 효과와 시너지 효과를 얻었습니다. 이는 코일 무게(단중) 증가와 같은 합병 효과를 통해 나타났습니다. 또한, 각 회사에서의 자재 구매를 통합함으로써 원가절감 효과를 얻었고, 하이스코의 15년 이상 축적한 냉연제품 생산 노하우를 공유함으로써 무형의 시너지를 얻었습니다.
    label:

스마트 팩토리 솔루션은 식품 제조업체와 철강업체에 각각 다른 방식으로 적용되고 있습니다.

식품 제조업체의 경우, 스마트 팩토리 솔루션은 주로 생산 관리와 설비 모니터링에 중점을 두고 있습니다. 예를 들어, 농축액 음료 제조업체 '진성에프엠'의 화성 공장에서는 에어릭스의 스마트 팩토리 IoT 솔루션이 도입되었습니다. 이 솔루션은 급기·배기 팬의 온도 및 진동을 모니터링하는 IoT 회전체 모니터링 시스템(iCMS)과 PLC 설비 모니터링 시스템을 포함하고 있습니다. 이를 통해 중앙 모니터링 시스템(iTMS)을 구축하여 관리자가 중앙에서 모든 설비의 이상 유무를 실시간으로 모니터링할 수 있게 되었습니다. 또한, 스마트폰이나 